In [2]:
import sqlalchemy

In [3]:
sqlalchemy.__version__ # check for drivers installed or not

'1.4.16'

In [4]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
engine = create_engine('sqlite:///college.db',echo=True) # creating engine , echo = true for logging , returns engine object
#dir(engine)
#engine.driver

In [5]:

meta = MetaData() # needed for creating tables in SQl . It holds a collection of Table objects as well as an optional binding to an Engine or Connection.
studs = Table(
   'studs',
    meta,
    Column('id',Integer,primary_key=True),
    Column('name',String),
    Column('lastname',String),
    ) #table creation
meta.create_all(engine) #creates all the tables in meta object in particular DB

2021-06-05 12:01:07,828 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-05 12:01:07,833 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("studs")
2021-06-05 12:01:07,834 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-05 12:01:07,836 INFO sqlalchemy.engine.Engine COMMIT


In [6]:

ins = studs.insert().values(name = 'Ravi', lastname = 'Kapoor')
ins1 = studs.insert().values(name = 'Pavan', lastname = 'Krishna')#insert operation
conn = engine.connect() # Before performing operation first need to connect to DB
conn.execute(ins)
conn.execute(ins1) # execute insert command

2021-06-05 12:01:09,146 INFO sqlalchemy.engine.Engine INSERT INTO studs (name, lastname) VALUES (?, ?)
2021-06-05 12:01:09,146 INFO sqlalchemy.engine.Engine [generated in 0.00076s] ('Ravi', 'Kapoor')
2021-06-05 12:01:09,148 INFO sqlalchemy.engine.Engine COMMIT
2021-06-05 12:01:09,202 INFO sqlalchemy.engine.Engine INSERT INTO studs (name, lastname) VALUES (?, ?)
2021-06-05 12:01:09,202 INFO sqlalchemy.engine.Engine [cached since 0.05599s ago] ('Pavan', 'Krishna')
2021-06-05 12:01:09,204 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
sel = studs.select()# select operation
res = conn.execute(sel) # returns cursor object
studs_res = res.fetchone()
studs_res = res.fetchall() # fetching the data

for res in studs_res:
    print(res)

2021-06-05 12:01:10,464 INFO sqlalchemy.engine.Engine SELECT studs.id, studs.name, studs.lastname 
FROM studs
2021-06-05 12:01:10,466 INFO sqlalchemy.engine.Engine [generated in 0.00140s] ()
(2, 'Pavan', 'Krishna')
(3, 'Ravi', 'Kapoor')
(4, 'Pavan', 'Krishna')
(5, 'Ravi', 'Kapoor')
(6, 'Pavan', 'Krishna')


In [8]:
sel = studs.select().where(studs.c.lastname=='Krishna')# select operation with where caluse
res = conn.execute(sel) # returns cursor object
studs_res = res.fetchall() # fetching the data

for res in studs_res:
    print(res)

2021-06-02 10:20:28,673 INFO sqlalchemy.engine.Engine SELECT studs.id, studs.name, studs.lastname 
FROM studs 
WHERE studs.lastname = ?
2021-06-02 10:20:28,674 INFO sqlalchemy.engine.Engine [generated in 0.00108s] ('Krishna',)
(2, 'Pavan', 'Krishna')
(4, 'Pavan', 'Krishna')


In [8]:
from sqlalchemy import text #another way of using select statement
t = text("SELECT * FROM studs")
result = conn.execute(t)
for res in result:
    print(res)

2021-06-05 12:01:12,232 INFO sqlalchemy.engine.Engine SELECT * FROM studs
2021-06-05 12:01:12,233 INFO sqlalchemy.engine.Engine [generated in 0.00103s] ()
(1, 'Ravi', 'Kapoor')
(2, 'Pavan', 'Krishna')
(3, 'Ravi', 'Kapoor')
(4, 'Pavan', 'Krishna')
(5, 'Ravi', 'Kapoor')
(6, 'Pavan', 'Krishna')


In [17]:
t = text("Select name,lastname from studs where lastname==:name")#passing values using parameters
result=conn.execute(t,name="Krishna").fetchall()
for res in result:
    print(res)

2021-06-02 10:34:30,800 INFO sqlalchemy.engine.Engine Select name,lastname from studs where lastname==?
2021-06-02 10:34:30,801 INFO sqlalchemy.engine.Engine [generated in 0.00122s] ('Krishna',)
('Pavan', 'Krishna')
('Pavan', 'Krishna')


In [9]:
from sqlalchemy.sql import select

s = select(text("name,lastname from studs")).where(text("lastname=='Krishna'")) #using text object in select and where caluse
#s = select([text("name,lastname from studs")]).where(studs.c.lastname=='Krishna')
result = conn.execute(s)
for res in result:
    print(res)

2021-06-05 12:01:14,585 INFO sqlalchemy.engine.Engine SELECT name,lastname from studs 
WHERE lastname=='Krishna'
2021-06-05 12:01:14,587 INFO sqlalchemy.engine.Engine [generated in 0.00113s] ()
('Pavan', 'Krishna')
('Pavan', 'Krishna')
('Pavan', 'Krishna')


In [10]:
from sqlalchemy import and_
s = select(text("name,lastname from studs")).where(
    and_(text("lastname=='Krishna'"),
       text("name=='Pavan'")))#using and_() method for adding multiple conditions in Where caluse
result = conn.execute(s)
for res in result:
    print(res)

2021-06-05 12:01:15,475 INFO sqlalchemy.engine.Engine SELECT name,lastname from studs 
WHERE lastname=='Krishna' AND name=='Pavan'
2021-06-05 12:01:15,475 INFO sqlalchemy.engine.Engine [generated in 0.00096s] ()
('Pavan', 'Krishna')
('Pavan', 'Krishna')
('Pavan', 'Krishna')


In [11]:
from sqlalchemy.sql import select
ali = studs.alias('st') # using alias
s = select(ali).where(ali.c.id>1)
conn.execute(s).fetchall()

2021-06-05 12:01:16,685 INFO sqlalchemy.engine.Engine SELECT st.id, st.name, st.lastname 
FROM studs AS st 
WHERE st.id > ?
2021-06-05 12:01:16,686 INFO sqlalchemy.engine.Engine [generated in 0.00107s] (1,)


[(2, 'Pavan', 'Krishna'),
 (3, 'Ravi', 'Kapoor'),
 (4, 'Pavan', 'Krishna'),
 (5, 'Ravi', 'Kapoor'),
 (6, 'Pavan', 'Krishna')]

In [12]:
#update operation
s = studs.update().where(studs.c.lastname =='Krishna').values(name = 'pavan k')
c = conn.execute(s)
s = studs.select()
res = conn.execute(s).fetchall()
for r in res:
    print(r)

2021-06-05 12:01:20,625 INFO sqlalchemy.engine.Engine UPDATE studs SET name=? WHERE studs.lastname = ?
2021-06-05 12:01:20,626 INFO sqlalchemy.engine.Engine [generated in 0.00093s] ('pavan k', 'Krishna')
2021-06-05 12:01:20,628 INFO sqlalchemy.engine.Engine COMMIT
2021-06-05 12:01:20,662 INFO sqlalchemy.engine.Engine SELECT studs.id, studs.name, studs.lastname 
FROM studs
2021-06-05 12:01:20,662 INFO sqlalchemy.engine.Engine [cached since 10.2s ago] ()
(1, 'Ravi', 'Kapoor')
(2, 'pavan k', 'Krishna')
(3, 'Ravi', 'Kapoor')
(4, 'pavan k', 'Krishna')
(5, 'Ravi', 'Kapoor')
(6, 'pavan k', 'Krishna')


In [13]:
#delete operation
s = studs.delete().where(studs.c.lastname=='Krishna')
c = conn.execute(s)
s = studs.select()
res = conn.execute(s).fetchall()
for r in res:
    print(r)

2021-06-05 12:07:56,114 INFO sqlalchemy.engine.Engine DELETE FROM studs WHERE studs.lastname = ?
2021-06-05 12:07:56,115 INFO sqlalchemy.engine.Engine [generated in 0.00104s] ('Krishna',)
2021-06-05 12:07:56,117 INFO sqlalchemy.engine.Engine COMMIT
2021-06-05 12:07:56,133 INFO sqlalchemy.engine.Engine SELECT studs.id, studs.name, studs.lastname 
FROM studs
2021-06-05 12:07:56,134 INFO sqlalchemy.engine.Engine [cached since 405.7s ago] ()
(1, 'Ravi', 'Kapoor')
(3, 'Ravi', 'Kapoor')
(5, 'Ravi', 'Kapoor')


In [21]:
#Relationship Between Tables
from sqlalchemy import ForeignKey,MetaData

meta = MetaData()
students = Table(
   'CollegeStudents', meta, 
   Column('id', Integer, primary_key = True), 
   Column('name', String), 
   Column('lastname', String), 
)
address = Table(
    'address',meta,
    Column('id',Integer,primary_key=True),
    Column('st_id',Integer,ForeignKey('CollegeStudents.id')),
    Column('postal_add', String), 
    Column('email_add', String)
)

meta.create_all(engine)

2021-06-05 12:24:57,507 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-05 12:24:57,508 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("CollegeStudents")
2021-06-05 12:24:57,509 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-05 12:24:57,510 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("CollegeStudents")
2021-06-05 12:24:57,511 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-05 12:24:57,512 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2021-06-05 12:24:57,512 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-05 12:24:57,513 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2021-06-05 12:24:57,514 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-05 12:24:57,515 INFO sqlalchemy.engine.Engine 
CREATE TABLE "CollegeStudents" (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	lastname VARCHAR, 
	PRIMARY KEY (id)
)


2021-06-05 12:24:57,515 INFO sqlalchemy.engine.Engine [no key 0.00048s] ()
2021-06-05 12:24:57,538 INFO sqlalchemy.engine.En

In [23]:
conn = engine.connect()
res=conn.execute(students.insert(),[{'name':'pk','lastname':'kp'},{'name':'pk1','lastname':'kp1'},{'name':'pavan','lastname':'kp'},{'name':'pk','lastname':'krishna'}])

2021-06-05 12:34:01,855 INFO sqlalchemy.engine.Engine INSERT INTO "CollegeStudents" (name, lastname) VALUES (?, ?)
2021-06-05 12:34:01,855 INFO sqlalchemy.engine.Engine [cached since 14.67s ago] (('pk', 'kp'), ('pk1', 'kp1'), ('pavan', 'kp'), ('pk', 'krishna'))
2021-06-05 12:34:01,858 INFO sqlalchemy.engine.Engine COMMIT


In [26]:
re = conn.execute(students.select()).fetchall()
for i in re:
    print(i)

2021-06-05 12:37:59,591 INFO sqlalchemy.engine.Engine SELECT "CollegeStudents".id, "CollegeStudents".name, "CollegeStudents".lastname 
FROM "CollegeStudents"
2021-06-05 12:37:59,592 INFO sqlalchemy.engine.Engine [cached since 85.91s ago] ()
(1, 'pk', 'kp')
(2, 'pk1', 'kp1')
(3, 'pavan', 'kp')
(4, 'pk', 'krishna')
(5, 'pk', 'kp')
(6, 'pk1', 'kp1')
(7, 'pavan', 'kp')
(8, 'pk', 'krishna')


In [27]:
conn.execute(address.insert(), [
   {'st_id':1, 'postal_add':'Shivajinagar Pune', 'email_add':'ravi@gmail.com'},
   {'st_id':1, 'postal_add':'ChurchGate Mumbai', 'email_add':'kapoor@gmail.com'},
   {'st_id':3, 'postal_add':'Jubilee Hills Hyderabad', 'email_add':'komal@gmail.com'},
   {'st_id':5, 'postal_add':'MG Road Bangaluru', 'email_add':'as@yahoo.com'},])
   

2021-06-05 12:38:44,274 INFO sqlalchemy.engine.Engine INSERT INTO address (st_id, postal_add, email_add) VALUES (?, ?, ?)
2021-06-05 12:38:44,275 INFO sqlalchemy.engine.Engine [generated in 0.00102s] ((1, 'Shivajinagar Pune', 'ravi@gmail.com'), (1, 'ChurchGate Mumbai', 'kapoor@gmail.com'), (3, 'Jubilee Hills Hyderabad', 'komal@gmail.com'), (5, 'MG Road Bangaluru', 'as@yahoo.com'))
2021-06-05 12:38:44,277 INFO sqlalchemy.engine.Engine COMMIT


In [37]:
s = students.select().where(students.c.id==address.c.st_id)
res = conn.execute(s).fetchall()
for i in res:
    print(i)

2021-06-05 12:54:15,371 INFO sqlalchemy.engine.Engine SELECT "CollegeStudents".id, "CollegeStudents".name, "CollegeStudents".lastname 
FROM "CollegeStudents", address 
WHERE students.name,students.lastname from students AND "CollegeStudents".id = address.st_id
2021-06-05 12:54:15,372 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ()
2021-06-05 12:54:15,372 INFO sqlalchemy.engine.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) near ",": syntax error
[SQL: SELECT "CollegeStudents".id, "CollegeStudents".name, "CollegeStudents".lastname 
FROM "CollegeStudents", address 
WHERE students.name,students.lastname from students AND "CollegeStudents".id = address.st_id]
(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [ ]:
,